In [8]:
import os
import math
import pandas as pd
import wget

## File paths
atlas = 'atlas.csv'
txtFile = 'listOfIds.txt'
fitsCSVs = 'fitsCSVs'

atlas = pd.read_csv(atlas)
starter = 'https://irsa.ipac.caltech.edu/applications/finderchart/servlet/api?mode=getImage&file_type=pdf&subsetsize=1.0&marker=true&POS=circle'

In [10]:
# Convert Panda series to arrays
objidA = pd.Series(atlas['objid']).array
raA = pd.Series(atlas['ra']).array
decA = pd.Series(atlas['dec']).array
ra, dec = 0, 0
id = '0'
grabbed = []
atlas

,objid,ra,dec
0,SSTSL2 J152742.67+023551.8,231.927823,2.597747
1,SSTSL2 J053049.72-670554.9,82.707184,-67.098598
2,SSTSL2 J153607.88+384214.4,234.032847,38.704015
3,SSTSL2 J224656.98+602924.5,341.737456,60.490152
4,SSTSL2 J022523.15-733726.6,36.346464,-73.624064
...,...,...,...
17459,SSTSL2 J053328.89-683636.5,83.370390,-68.610158
17460,SSTSL2 J194613.16+263840.3,296.554874,26.644537
17461,SSTSL2 J193929.88+233823.8,294.874522,23.639946
17462,SSTSL2 J174449.11-333318.8,266.204645,-33.555244


In [9]:
# Asks for Object ID, then returns ra and dec of the object is available
# If not available, display error and ends program
# Edit: replaces spaces in ID with underscores
def search(id):
    for i in range(len(objidA)):
        if id == objidA[i]:
            ra = raA[i]
            dec = decA[i]
            print(f'Ra={ra}, Dec={dec}')
            id = id.replace(' ', '_')
            return (id, ra, dec)
    print(f'Error: search()\nInvalid id: {id}')
    exit()

# Rounds input to the 5th decimal and outputs string
def roundCords(num):
    temp = (num*100000)+0.5
    temp = math.floor(temp)
    temp = str(float(temp)/100000)
    return(temp)

# Outputs website
def makeWeb(r, d):
    endP = f'{starter}+{r}+{d}'
    print(f'Website:\n{endP}')
    return(endP)

# downloads file, if not already downloaded
def grab(id):
    id, ra, dec = search(id)
    website = (makeWeb(roundCords(ra),roundCords(dec)))
    
    # adds all file names in 'results' in a list
    results = []
    for file_path in os.listdir(fitsCSVs):
        if os.path.isfile(os.path.join(fitsCSVs, file_path)):
            results.append(file_path)

    #checks if the csv is already downloaded
    if (id+'.csv') in results:
        print(f'{id}.csv already in fitsCSVs folder\n')
    else:
        print('Grabbing...')
        wget.download(website, f'{id}.csv')
        os.rename(f'{id}.csv', f'{fitsCSVs}/{id}.csv')
        print(f'\nDownloaded: {id}.csv\n')

# takes ids in listOfIds.txt into a list
def listOut():
    list = []
    myfile = open(txtFile, "rt")
    contents = myfile.read() + '\n'
    myfile.close()
    trys = 0
    while len(contents) > 0:
        index = contents.find('\n')
        id = contents[0:index]
        list.append(id)
        contents = contents[index+1:]
    return(list)


In [ ]:
# ask = input('Single search? (y/n)\n')
# if ask == 'y':
#     id = input('Enter object ID: ')
#     grab(id)
# elif ask == 'n':
listOfIds = listOut()
if len(listOfIds) > 0:
    for obj in range(len(listOfIds)):
        grab(listOfIds[obj])
else:
    print('No ids in listOfIds.txt')
# else:
#     print('Invalid input')